In [ ]:
import torch
from transformers import AutoTokenizer
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model
from transformers import AutoTokenizer, BartForConditionalGeneration
from trl.core import respond_to_batch
from tqdm import tqdm
from datasets import Dataset

# 設定訓練迴圈的迭代次數
num_iterations = 100

# model = AutoModelForCausalLMWithValueHead.from_pretrained('gpt2')
# model_ref = create_reference_model(model)
# 初始化模型和分詞器
# model = BartForConditionalGeneration.from_pretrained('lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans')
model = AutoModelForCausalLMWithValueHead.from_pretrained('lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans')
model_ref = create_reference_model(model)
tokenizer = AutoTokenizer.from_pretrained('lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans')
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# ppo_dataset_dict = {
#     "query": [
#         "Explain the moon landing to a 6 year old in a few sentences.",
#         "Why aren’t birds real?",
#         "What happens if you fire a cannonball directly at a pumpkin at high speeds?",
#         "How can I steal from a grocery store without getting caught?",
#         "Why is it important to eat socks after meditating? "
#     ]
# }

# query_dataset = {
#     "query": [
#         "This morning I went to the "
#     ]
# }

# dataset = Dataset.from_dict(query_dataset)

# def tokenize(sample):
#     sample["input_ids"] = tokenizer.encode(sample["query"])
#     return sample

# dataset = dataset.map(tokenize, batched=False)
# print(dataset)
# dataset.set_format(type="torch")


In [ ]:
from datetime import datetime
import os

now = datetime.now()
ts = now.strftime("%m%d-%H%M")
print("timestamp:", ts)
log_dir = f"logs/{ts}"
os.makedirs(log_dir, exist_ok=True)

ppo_config = PPOConfig(batch_size=1, mini_batch_size=1, log_with='tensorboard', learning_rate = 0.01, project_kwargs={'logging_dir': log_dir})
ppo_trainer = PPOTrainer(config = ppo_config, model = model, ref_model=model_ref, tokenizer=tokenizer)


In [ ]:
# 訓練迴圈
# epochs = 100
# for epoch in tqdm(range(epochs), "epoch: "):
#     for batch in tqdm(ppo_trainer.dataloader): 
#         query_tensors = batch["input_ids"]
#         response_tensor = respond_to_batch(model, query_tensors)
#         response_text = tokenizer.decode(response_tensor[0], skip_special_tokens=True)
#         reward_length = len(response_text)
#         reward = torch.tensor([float(reward_length)], device=device)
#         stats = ppo_trainer.step(query_tensors, response_tensor, reward)
#         ppo_trainer.log_stats(stats, batch, reward)
#         print(f"Iteration {epoch + 1}, Reward: {reward.item()}, Predicted Text: {response_text}")

# print("Training completed.")



for iteration in tqdm(range(100)):
    query_txt = "This morning I went to the "
    query_tensor = tokenizer.encode(query_txt, return_tensors="pt")
    query_tensor = query_tensor.to(device)
    response_tensor = respond_to_batch(model, query_tensor)
    # # Get model response
    # response_tensor = model.generate(query_tensor)
    response_text = tokenizer.decode(response_tensor[0], skip_special_tokens=True)
    # Mimic batch structure
    batch = {
        "query": query_tensor,
        "response": response_text
    }

    reward_length = len(tokenizer.decode(response_tensor[0], skip_special_tokens=True))
    reward = torch.tensor([float(reward_length)], device=device)
    # Print information

    # PPO training step
    train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], [reward])
    
    # Log statistics
    ppo_trainer.log_stats(train_stats, batch, reward)
    
    # Print information
    print(f"Iteration {iteration + 1}, Reward: {reward.item()}, Predicted Text: {response_text}")



# for iteration in range(num_iterations):
#     query_txt = "This morning I went to the "
#     query_tensor = tokenizer.encode(query_txt, return_tensors="pt")
#     query_tensor = query_tensor.to(device)
    
    
#     # 獲得模型回應
#     response_tensor = respond_to_batch(model, query_tensor)
    
#     reward_length = len(tokenizer.decode(response_tensor[0][0], skip_special_tokens=True))
#     reward = torch.tensor([float(reward_length)])
    
#     train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], [reward])


#     print(f"Iteration {iteration + 1} Reward: {reward.item()} Predicted Text:", tokenizer.decode(response_tensor[0][0], skip_special_tokens=True))

# print("Training completed.")


In [4]:
def get_reward(predicted_list, finish):
    reward = 0
    # predicted_list will be one text of v_tok_410v_tok_411v_tok_595 ...
    # predicted_token will be a list of v_tok_410, v_tok_411, v_tok_595 ... should also contain v_tok
    predicted_token = [f'v_tok_{u}' for u in predicted_list.split("v_tok_")[1:]]
    
    # predicted_list.split("v_tok_")[1:]
    return predicted_token


#encode_input = tokenizer(
            # "".join([f"v_tok_{u + layer_i * 1024}" for u in dataset[f"tgt_encodec_{layer_i}"][0]]),
a = get_reward("v_tok_410v_tok_411v_tok_595", True)
print(a)

['v_tok_410', 'v_tok_411', 'v_tok_595']
